In [5]:
def resize_with_aspect_ratio(img, new_width=None, new_height=None):
    # Get the current height and width
    height, width = img.shape[:2]

    # If only width is specified
    if new_width is not None and new_height is None:
        # Calculate the aspect ratio and new height
        aspect_ratio = width / height
        new_height = int(new_width / aspect_ratio)

    # If only height is specified
    elif new_height is not None and new_width is None:
        # Calculate the aspect ratio and new width
        aspect_ratio = height / width
        new_width = int(new_height / aspect_ratio)

    # If both width and height are specified, ignore aspect ratio
    elif new_width is not None and new_height is not None:
        pass

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height))
    return resized_img

In [6]:
def draw_landmarks(image, landmarks):

    radius = 5
    # Check if image width is greater than 1000 px.
    # To improve visualization.
    print('*********** landmarks: ', landmarks)
    
    if (image.shape[1] > 1000):
        radius = 8

    for idx, kpt_data in enumerate(landmarks):

        # loc_x, loc_y = kpt_data[:2].astype("int").tolist()
        loc_x, loc_y = kpt_data[:2]
        
        color_id = list(COLORS_RGB_MAP[int(kpt_data[-1])].values())[0]

        cv2.circle(image,
                   (loc_x, loc_y),
                   radius,
                   color=color_id[::-1],
                   thickness=-1,
                   lineType=cv2.LINE_AA)
         # Draw keypoint number
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1
        # text = str(int(kpt_data[-1]))
        text = f"{int(kpt_data[-1])}: ({loc_x}, {loc_y})"
        text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
        text_x = loc_x - text_size[0] // 2
        text_y = loc_y - radius - 5
        print('************* text:', text)
        cv2.putText(image, text, (text_x, text_y), font, font_scale, color=(255, 255, 255), thickness=font_thickness)


    return image


In [7]:
def wear_collar(accessories_img,animal_image,  filter_kpts):
    ############################### cap working ############################### 
    # get value of 8 and 2
    point_8 = next(sublist for sublist in filter_kpts if sublist[2] == 8)
    point_2 = next(sublist for sublist in filter_kpts if sublist[2] == 2)
    print('point_8 and point_2', point_8, point_2 )

    ### check if point 8 and point 2 exits 
    if point_8 and point_2:
        print('exits+++++++++++++++++')

        #### width between 8 and 2
        width_8_2 = np.sqrt((point_2[0] - point_8[0])**2 + (point_2[1] - point_8[1])**2)

        #### resize image according to width if the 8 and 2 points 
        resized_accessories_img  = resize_with_aspect_ratio(accessories_img, int(width_8_2))

        # Calculate the midpoint
        midpoint_x = int((point_2[0] + point_8[0]) / 2)
        midpoint_y = int((point_2[1] + point_8[1]) / 2)
        print("&&&&&&&& mid between 8 and 2 midpoint_x  and midpoint_y", midpoint_x, midpoint_y)

        
        point_17 = next(sublist for sublist in filter_kpts if sublist[2] == 17)
        
        mid_17_bet_width_8_2 = np.sqrt((midpoint_x - point_17[0])**2 + (midpoint_y - point_17[1])**2)
        print('width between mid and 17: ', mid_17_bet_width_8_2)

        if mid_17_bet_width_8_2 <= 100:
            percentage_to_subtract = 35
            offset_y = int(percentage_to_subtract / 100 * resized_accessories_img.shape[1])
            image = cvzone.overlayPNG(animal_image, resized_accessories_img, (point_8[0] , point_8[1] - offset_y))
            
        else:
            midpoint_x_17 = int((midpoint_x + point_17[0]) / 2)
            midpoint_y_17 = int((midpoint_y + point_17[1]) / 2)
            print("^^^^^^^^^^^^^^^^ mid between mid and 7 ", midpoint_x_17, midpoint_y_17)
            
            ######## show point in specific location
            percentage_to_subtract = 45
            offset_y = int(percentage_to_subtract / 100 * resized_accessories_img.shape[1])
    
            
            
            image = cvzone.overlayPNG(animal_image, resized_accessories_img, (midpoint_x_17 - offset_y, midpoint_y_17 - offset_y))
        return image        
        # cv2.imwrite("resized_accessories_img_check.jpg", image)
        # cv2.imshow("resized_accessories_img", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
            
        
        
    else:
        missing_points = [point for point in points_to_check if point not in [sublist[2] for sublist in filter_kpts]]
        print(f" ************The following points are missing: {missing_points}")
    ############################### cap working end ###############################

In [10]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
import os
import numpy as np
import pandas as pd
import cvzone
from PIL import Image 

model_path = 'animal-tryon/best.pt'
keywords_file = "animal-tryon/keypoint_definitions.csv"
# animal_img = 'animal-tryon/input/HD-wallpaper-dog-ears-sitting-big.jpg'
# animal_img = 'animal-tryon/images_data/frame_56.png'
animal_img = 'animal-tryon/input/dog1.jpg'

# accessories_img = 'animal-tryon/accessories/collar-edited.png'
# result_png = 'png_result.png'
accessories_img_path = "animal-tryon/accessories/edited_images/12 I'm So Hungry/pet-light-red-cropped.png"
result_png = "2. 12 I'm So Hungry(Product result).png"

ann_meta_data = pd.read_csv(keywords_file)
COLORS = ann_meta_data["Hex colour"].values.tolist()

COLORS_RGB_MAP = []
for color in COLORS:
    R, G, B = int(color[:2], 16), int(color[2:4], 16), int(color[4:], 16)
    COLORS_RGB_MAP.append({color: (R,G,B)})

model = YOLO(model_path)

BOX_IOU_THRESH = 0.55
BOX_CONF_THRESH=0.30
KPT_CONF_THRESH=0.68

inc = 15



animal_image = cv2.imread(animal_img)
animal_image = cv2.cvtColor(animal_image, cv2.COLOR_BGR2RGB)

accessories_img = cv2.imread(accessories_img_path, cv2.IMREAD_UNCHANGED)


results = model.predict(animal_img, conf=BOX_CONF_THRESH, iou=BOX_IOU_THRESH)[0].cpu()

if not len(results.boxes.xyxy):
    animal_image

# Get the predicted boxes, conf scores and keypoints.
pred_boxes = results.boxes.xyxy.numpy()
pred_box_conf = results.boxes.conf.numpy()
pred_kpts_xy = results.keypoints.xy.numpy()
pred_kpts_conf = results.keypoints.conf.numpy()

print('results.boxes.xyxy.numpy()', results.boxes.xyxy.numpy())



# Draw predicted bounding boxes, conf scores and keypoints on image.
for boxes, score, kpts, confs in zip(pred_boxes, pred_box_conf, pred_kpts_xy, pred_kpts_conf):
    kpts_ids = np.where(confs > KPT_CONF_THRESH)[0]
    filter_kpts = kpts[kpts_ids]
    filter_kpts = np.concatenate([filter_kpts, np.expand_dims(kpts_ids, axis=-1)], axis=-1)
    
    # filter_kpts = filter_kpts.astype("int").tolist()
    filter_kpts = [[int(x) for x in inner_list] for inner_list in filter_kpts]
    print('******** old filter_kpts: ', filter_kpts)
    # animal_image = draw_landmarks(animal_image, filter_kpts)

    animal_image = cv2.cvtColor(animal_image, cv2.COLOR_BGR2RGB)
    # animal_image = draw_landmarks(animal_image, filter_kpts)
    
    img_result = wear_collar(accessories_img, animal_image, filter_kpts)
    
    # cv2.imshow("resized_accessories_img", animal_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    
    # Check if the resulting image has valid dimensions
    if img_result is not None and img_result.shape[0] > 0 and img_result.shape[1] > 0:
        cv2.imwrite(result_png, img_result) 
        cv2.imshow("cap_wear_result", img_result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Invalid dimensions for the resulting image.")

        # print("********************* Neither 14 nor 15 exists.")
    ################ 


image 1/1 D:\closet\tryon\animal-tryon\input\dog1.jpg: 640x448 1 dog, 516.7ms
Speed: 2.0ms preprocess, 516.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 448)
results.boxes.xyxy.numpy() [[         89          80         587         926]]
******** old filter_kpts:  [[432, 906, 0], [402, 794, 1], [391, 613, 2], [202, 895, 6], [224, 785, 7], [227, 606, 8], [423, 164, 14], [304, 147, 15], [343, 322, 16], [341, 378, 17], [450, 94, 18], [294, 73, 19]]
point_8 and point_2 [227, 606, 8] [391, 613, 2]
exits+++++++++++++++++
&&&&&&&& mid between 8 and 2 midpoint_x  and midpoint_y 309 609
width between mid and 17:  233.20591759215716
^^^^^^^^^^^^^^^^ mid between mid and 7  325 493
